In [10]:
import os
import sqlite3
import json
import random
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from tqdm.notebook import tqdm

import torch
import numpy as np
import datasets
import transformers
import wandb
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback, DataCollator

import sys
sys.path.append('../../')
from subset_active_learning.subset_selection import select, preprocess

from sklearn.decomposition import PCA

In [34]:
db_name = '../sst_results_diversity_annealing.db'
seed = 0
n_downsample = 1000
wandb_project = 'sst_search_test'
wandb_entity = 'johntzwei'

ds_name = 'sst2'
test_dataset = None
eval_mapping = '[]'
num_labels = 2
valid_split = 'validation'
test_split = 'test'

model_card = "roberta-base"
pretraining = True
max_steps = 1000
eval_steps = 300
learning_rate = 1e-5
batch_size = 8
# adam should default to correct_bias = True
adam_epsilon = 1e-6
adam_beta1 = 0.9
adam_beta2 = 0.999
max_grad_norm = 1.0
warmup_ratio = 0.0
# weight_decay = 0.01
weight_decay = 0.3

In [28]:
training_args = select.SubsetTrainingArguments(model_card=model_card,
                                               num_labels=num_labels,
                                               eval_mapping=json.loads(eval_mapping),
                                               pretraining=pretraining,
                                               max_steps=max_steps,
                                               eval_steps=eval_steps,
                                               learning_rate=learning_rate,
                                               batch_size=batch_size,
                                               adam_epsilon=adam_epsilon,
                                               adam_beta1=adam_beta1,
                                               adam_beta2=adam_beta2,
                                               max_grad_norm=max_grad_norm,
                                               warmup_ratio=warmup_ratio,
                                               weight_decay=weight_decay)

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [13]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1080 Ti'

In [16]:
tokenized_sst2 = preprocess.get_dataset(ds_name, model_card)

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-53804d3571838bad.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6257b67c9611994c.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-69a14aade0efa57f.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-4191b237bd8001be.arrow
Loading cached processed dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-ae483633eb1f52ef.arrow
Loading cached processed dataset at /home/johnny/.cache

In [18]:
sst2_downsample = tokenized_sst2['train'].shuffle(seed=seed).select(range(0, n_downsample))
sst2_remaining = tokenized_sst2['train'].shuffle(seed=seed).select(range(n_downsample, len(tokenized_sst2['train'])))

Loading cached shuffled indices for dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-8e3ec244989c9f34.arrow
Loading cached shuffled indices for dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-8e3ec244989c9f34.arrow


In [19]:
val_dataloader = torch.utils.data.DataLoader(tokenized_sst2['validation'], shuffle=False, batch_size=batch_size, pin_memory=True)
test_dataloader = torch.utils.data.DataLoader(tokenized_sst2['test'], shuffle=False, batch_size=batch_size, pin_memory=True)

In [20]:
def create_hf(model_card, num_labels=2):
    tokenizer = AutoTokenizer.from_pretrained(model_card)
    model = AutoModelForSequenceClassification.from_pretrained(model_card, num_labels=num_labels)
    return model, tokenizer

In [24]:
model, tokenizer = create_hf(model_card, num_labels=2)
model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [30]:
subset_trainer = select.SubsetTrainer(
    params=training_args, valid_ds=processed_ds[valid_split], test_ds=processed_ds[test_split]
)

In [35]:
wandb.init(project=wandb_project, entity=wandb_entity, tags=[model_card])
quality = subset_trainer.train_one_step(sst2_downsample)

wandb: Currently logged in as: johntzwei (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.13.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2022-10-17 15:36:09.259408: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-17 15:36:09.259444: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

  0%|          | 0/138 [00:00<?, ?it/s]

In [26]:
train_dataloader = torch.utils.data.DataLoader(tokenized_sst2['train'], shuffle=False, batch_size=batch_size, pin_memory=True)
embeddings = evaluate(model, train_dataloader, score=True, output_hidden_states=True)['hidden_states'].cpu().numpy()
embs = torch.nn.Embedding.from_pretrained(torch.Tensor(embeddings))
torch.save(embs, 'trained-emb_train.pt')

  0%|          | 0/1068 [00:00<?, ?it/s]

/tmp/ipykernel_23643/1496236283.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  batch_scores = torch.nn.Softmax()(logits)[:,-1]


In [44]:
with_idx = processed_ds['train'].add_column('idx', range(len(processed_ds['train'])))
with_idx = with_idx.shuffle(seed=seed)
np.save('shuffled_idx.npz', with_idx['idx'].numpy())

Loading cached shuffled indices for dataset at /home/johnny/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-9a07259d62c63984.arrow
